
### Implementation of sparse coding and learning with different regularizers
#### Regularizers: ISTA, Hard thresholding, $\lambda_{1/2}$ thresholding, and CEL0

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import utils
from generativeModels import SparseModel, runModelSim


### Functions used for plotting

In [2]:
#gets the Phis from the model and draws them in 2-d numUnitsXnumUnits
def plotReceptiveFields(Phi, numUnits, sz):

    # Plot Receptive fields
    fig = plt.figure(figsize=(20,20))
    plt.subplots_adjust(hspace=0.1, wspace=0.1)
    columns = int(np.floor(numUnits/25))
    rows = int(np.floor(numUnits/columns))
    for i in range(numUnits):
        plt.subplot(rows, columns, i+1)
        plt.imshow(np.reshape(Phi[:, i], (sz, sz)), cmap="gray")
        plt.axis("off")

    fig.suptitle("Receptive fields", fontsize=20)
    plt.subplots_adjust(top=0.9)
    #plt.show()


In [3]:
#Plots the error as a function of iterations
def plotError(errorList, color = 'r' ):

    # Plot error
    plt.figure(figsize=(5, 3))
    plt.ylabel("Error")
    plt.xlabel("Iterations")
    plt.plot(np.arange(len(errorList)), np.array(errorList),color = color)
    plt.tight_layout()
    plt.show() 

## Generative model with thresholding function

#### Define the network implemented above as a model. Also, initialize the constants used in the simulation.

#### *Change the variable `flagMethod` depending on what thresholding function you want to use: `soft`,`hard1`,`half`, or `CEL0` 
#### Change the variable `numUnits` depending on how many units ($\phi$ vectors) you wish to have

#### `batchSize` indicates the number of input patches that are processed at once to produce (for them) sparse codes and at the end adjust the $\phi$ vectors (do the learning)

#### `numIter` indicates the number of different batches you use. `numRSeqs = numIter*batchSize` is the number of input image patches we need 

In [4]:
flagMethod = 'soft' #flagMethod = 'hard1';flagMethod = 'half';flagMethod = 'CEL0'
numUnits = 500# number of neurons (units)


#number of input patches = number of iterationsXbatch size
numIter = 4000; batchSize = 250; numRSeqs = numIter*batchSize 
sz = 16 # image patch size

# Define model
lmda = {};lrR = {}
lmda['soft'] = 4.1e-1; lrR['soft'] = 1e-2
lmda['hard1'] = 1.3e-2; lrR['hard1'] = 1e-2
lmda['half'] = 1.3e-1; lrR['half'] = 1e-2
lmda['CEL0'] = 4.5e-1; lrR['CEL0'] = 1e-1

# Define model
model = SparseModel(sz**2,numUnits, batchSize,lmda[flagMethod],flagMethod ,lrR[flagMethod])


#Load the images
fileLoad = 'data/DoGPatchesScaleShift.pckl'
#Get the whitened cropped imagess that we are going to use for all algorithms
inputsAll = utils.loadVar(fileLoad)
#the initial r's are all zero
rInitZeros = np.zeros((numRSeqs,numUnits))


#Run the model for all the input image patches
model,rAll_, errorList = runModelSim(model,numIter,batchSize,inputsAll,rInitZeros)

  0%|          | 0/4000 [00:00<?, ?it/s]

iter: 100/4000, Moving error: 0.027646212070531456


KeyboardInterrupt: 

In [ ]:
#To save the model, phi vectors, r sequences
fileSave = 'data/rPhiModel'+flagMethod+'.pckl'

rAll = np.array(rAll_); Phi = model.Phi

utils.saveVar((errorList,rAll,Phi,model),fileSave)


### Drawing loss 


In [ ]:
plotError(errorList)

### Drawing the columns of $\Phi$  (receptive field)


In [ ]:
plotReceptiveFields(model.Phi, numUnits, sz)
